In [2]:
%matplotlib ipympl
from QDP import qdp
import numpy as np
import matplotlib.pyplot as plt
import os.path
dp = '/home/ebert/jupyter-notebooks/FNode-data'

In [44]:
exp_name = '2018_04_30_17_11_26_cs-rb-load-collision-test'
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)
print "ivars: {}".format(q.experiments[0]['variable_list'])
roi_map = ['cs', 'rb']
cs_cuts = [[31],[2]]
rb_cuts = [[42],[3]]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
retention = q.apply_thresholds()
tbl_str = "ivar:\t{0}\tretention:{1} +- {2}"
entries = np.product(retention['retention'].shape[:-1])
for i in range(entries):
    print(tbl_str.format(
        retention['ivar'].reshape((entries,-1))[i][0],
        retention['retention'].reshape((entries,-1))[i],
        retention['error'].reshape((entries,-1))[i]
    ))
# q.save_experiment_data()
q.experiments[0]['variable_list']
try:
    xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']
except IndexError:
    print("Only one iteration")
    xlab = ''

fig, ax = plt.subplots()
for e in range(len(retention['ivar'])):
    for r in range(2):
        ax.errorbar(
            retention['ivar'][e, :, r], retention['retention'][e, :, r],
            yerr=retention['error'][e, :, r],
            fmt=['bo','rx'][r]
        )
ax.set_ylim(0, 1.01)
ax.set_xlabel(xlab)
ax.set_ylabel('F=4(2) pop.')

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'retention_scan.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

problem analyzing timeseries data in iteration.
ivars: []
ivar:	0.0	retention:[ 0.85115601  0.74521073] +- [        nan  0.01134164]
Only one iteration


FigureCanvasNbAgg()

In [47]:
qd = q.experiments[0]['iterations'][0]['quantized_data']

In [53]:
dual_load = np.logical_and(qd[:,0,0], qd[:,0,1])
cs_only = np.logical_and(qd[:,0,0], np.logical_not(qd[:,0,1]))
rb_only = np.logical_and(np.logical_not(qd[:,0,0]), qd[:,0,1])

In [54]:
print(np.mean(qd[dual_load,1], axis=0))
print(np.mean(qd[cs_only,1], axis=0))
print(np.mean(qd[rb_only,1], axis=0))

[ 0.77419355  0.60645161]
[ 0.85164208  0.00541928]
[ 0.01349831  0.76940382]


In [51]:
sum(dual_load)

155